## Temperature forecasting for different cities in the world

## Introduction
For this project you are asked to analyze three datasets, called respectively:
1. pollution_us_2000_2016.csv
2. greenhouse_gas_inventory_data_data.csv
3. GlobalLandTemperaturesByCity.csv

You are asked to extract from dataset 2 only the US countries (for which we have info in the other datasets) and to perform the following tasks:
- to measure how pollution and temperature create cluster tracing the high populated cities in the world
- to analyze the correlation between pollution data and temperature change.
- to predict the yearly temperature change of a given city over a given time period, using the <b>ARIMA model</b> for <b>time series forecasting</b>, that is a model for time series forecasting integrating AR models with Moving Average.
- (OPTIONAL) rank the 5 cities that will have a highest temperature change in US


### TASK1 :Cluster Analysis
You use K-means or DBSCAN to perform the cluster analysis, and create a new dataset where the cities are associated to the different identified clusters

### TASK 2: Correlation Analysis

You measure the correlation between:
- temperature and latitude
- temperature and pollution
- temperature change (difference between the average temperature measured over the last 3 years and the previous temperature) and pollution


### TASK 3: Predicting the Temperature of a Given City across a Specified Time Period
After reading the data in the temperature data set, for each city cluster, before applying the ARIMA model you perform the following steps:

- EDA
- data cleaning and preprocessing (Converting the 'dt' (date) column to DateTime format, removing NaN)
- feature selection
- make the time-series stationary
- check for stationarity : Calculating the Augmented Dickey-Fuller Test statistic 
- identify the (p, q) order of the ARIMA model using ACF partial autocorrelation plot

Then:

-fit the ARIMA model using the calculated p, q values.
-calculate the MSE with respect to the true temp. measurements to estimate the performance of the model


NOTE: ARIMA models need the data to be stationary i.e. the data must not exhibit trend and/or seasonality. To identify and remove trend and seasonality, we can use
- seasonal decomposition
- differencing

In [2]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA, ARMAResults
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets


import seaborn as sns

from gustavo_functions import *

%load_ext autoreload
%autoreload 2


## SECTION 1: Cluster Analysis

In [ ]:
# read the csv file containing the polluters 
df_pollution = pd.read_csv#("cata/pollution_us_2000_2016.csv")

In [2]:
# read the csv file containing temperature data into a DataFrame
df_temp = pd.read_csv("data/GlobalLandTemperaturesByCity.csv")

city_df #city dataset with cluster labels

## SECTION 2: Correlation Analysis

##SECTION 3: ARIMA model for temperature forecasting

## Profiling Report 

In [ ]:
# 
filepath = "data/Euro_2012_stats_TEAM.csv"
euro12 = pd.read_csv(filepath)

Hello Gustavo!


In [ ]:
profile = ProfileReport(euro12,title="Euro 12 Profiling Report")

In [ ]:
profile.to_notebook_iframe()

In [ ]:
# To save the profile but in this case I dont'n want to save 
#profile.to_file("report.html")

## Conclusion

write here the report for the project